# Importation des Librairies + Datasets

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration pour afficher tous les graphiques dans le notebook
%matplotlib inline
sns.set_theme(style="whitegrid")

In [4]:
# Chargement des 4 piliers du projet
objects = pd.read_csv('objects.csv', encoding='ISO-8859-1')
funding_rounds = pd.read_csv('funding_rounds.csv', encoding='ISO-8859-1')
investments = pd.read_csv('investments.csv', encoding='ISO-8859-1')
acquisitions = pd.read_csv('acquisitions.csv', encoding='ISO-8859-1')

ParserError: Error tokenizing data. C error: EOF inside string starting at row 16826

# Analyse des fichiers

In [ ]:
# Dictionnaire des DataFrames pour automatiser l'analyse
datasets = {
    "Objects": objects,
    "Funding Rounds": funding_rounds,
    "Investments": investments,
    "Acquisitions": acquisitions
}

for name, df in datasets.items():
    print(f"\n{'='*20} {name} {'='*20}")
    # Analyse de la structure [cite: 350]
    print(f"Dimensions : {df.shape}")
    print("\n--- Infos structurelles ---")
    df.info()
    # Détection des valeurs manquantes [cite: 352]
    print("\n--- Valeurs manquantes ---")
    print(df.isnull().sum())
    # Visualisation d'un échantillon [cite: 347]
    print("\n--- Échantillon (10 lignes) ---")
    display(df.head(10))

## 1. Analyse Objects.csv

- Plus de 73% de manquants pour category_code et 78% pour founded_at

- De nombreuses colonnes comme parent_id, logo_url ou domain n'ont aucune valeur prédictive pour le succès d'une startup

## 2. Analyse funding_rounds.csv

- Contrairement au fichier Objects, la colonne cruciale raised_amount_usd ne présente aucune valeur manquante (52 928 non-null). C'est une excellente nouvelle pour la précision de votre futur modèle supervisé.

- Les colonnes de valorisation (pre_money_valuation et post_money_currency_code) sont vides à près de 50%. Elles seront donc écartées pour éviter de réduire drastiquement la taille du dataset final.

- La date de levée (funded_at) est quasi complète (seulement 248 manquants sur 53k lignes), ce qui permettra de calculer la vélocité de financement.

## 3. Analyse investments.csv

- Ce fichier est techniquement "parfait" avec 0 valeur manquante sur les 80 902 lignes.

- Il contient les clés de liaison fondamentales : funded_object_id (la startup) et investor_object_id (l'investisseur)

- Il s'agit d'une table de faits qui répertorie chaque participation d'un investisseur à un round de financement.

## 4. Analyse acquisitions.csv

- Ce fichier identifie les cibles (acquired_object_id) et les acheteurs (acquiring_object_id).

- La structure est relativement saine pour les colonnes d'identification. Cependant, les colonnes financières (price_amount) et contractuelles (term_code) sont très lacunaires ou comportent des valeurs à zéro.

- Le term_code est manquant à 80 % (7 656 vides). Comme pour les fichiers précédents, nous allons écarter les colonnes "bruit" (URLs, descriptions) pour optimiser la mémoire.


# Phase de Nettoyage

## 1.Nettoyage objects.csv

In [ ]:
# 1. Filtrage par entité (Action métier obligatoire)
# On ne garde que les entreprises (Company) pour cibler la problématique
df_objects_clean = objects[objects['entity_type'] == 'Company'].copy()

# 2. Sélection des colonnes stratégiques (Réduction de dimensionnalité)
# On supprime les colonnes inutiles ou trop vides pour optimiser la mémoire
cols_to_keep = [
    'id', 'name', 'category_code', 'status', 'founded_at',
    'country_code', 'funding_total_usd', 'funding_rounds', 'relationships'
]
df_objects_clean = df_objects_clean[cols_to_keep]

# 3. Traitement des valeurs manquantes
# On supprime les lignes sans secteur, pays ou date (trop critiques pour être imputées)
df_objects_clean.dropna(subset=['category_code', 'founded_at', 'country_code'], inplace=True)

# 4. Standardisation des types
# Conversion de la date de fondation en objet datetime
df_objects_clean['founded_at'] = pd.to_datetime(df_objects_clean['founded_at'], errors='coerce')

# 5. Création de la variable Target
# Transformation du statut en binaire : 1 si succès (acquired/ipo), 0 sinon
df_objects_clean['is_success'] = df_objects_clean['status'].apply(lambda x: 1 if x in ['acquired', 'ipo'] else 0)

print(f"Volume après nettoyage : {df_objects_clean.shape}")

- Un modèle ne peut être meilleur que la donnée fournie. En supprimant les lignes incomplètes sur les secteurs et les dates, on assure des résultats généralisables

- Les algorithmes de ML ne traitent pas les dates brutes ; la conversion est un prérequis obligatoire pour l'ingénierie de variables futures.

- Cette approche réduit le volume global mais augmente la densité d'information, ce qui est le plus important pour notre scoring

## 2. Nettoyage funding_rounds.csv

In [ ]:

# Sélection des colonnes utiles (Réduction de la dette technique)
cols_finance = ['object_id', 'funding_round_type', 'raised_amount_usd', 'funded_at']
df_rounds_clean = funding_rounds[cols_finance].copy()

# Suppression des rares lignes sans date
df_rounds_clean.dropna(subset=['funded_at'], inplace=True)

# Conversion au format Datetime (
df_rounds_clean['funded_at'] = pd.to_datetime(df_rounds_clean['funded_at'], errors='coerce')

# Aperçu statistique des montants (Détection d'outliers)
print("Statistiques des levées (USD) :")
print(df_rounds_clean['raised_amount_usd'].describe())

print(f"\nVolume final de df_rounds_clean : {df_rounds_clean.shape}")

- On ne conserve que la "moelle épinière" financière : l'ID de la startup, le type de round, le montant en USD et la date.

- Conversion de funded_at en format datetime pour permettre les calculs chronologiques.

- S'assurer que les montants sont bien traités comme des numériques continus pour le futur scaling.

## 3. Nettoyage investments.csv

In [ ]:

# Sélection des colonnes relationnelles
df_investments_clean = investments[['funding_round_id', 'funded_object_id', 'investor_object_id']].copy()

# Détection et suppression des doublons
duplicate_count = df_investments_clean.duplicated().sum()
print(f"Nombre de doublons détectés : {duplicate_count}")
df_investments_clean.drop_duplicates(inplace=True)

# Vérification de la structure
print(f"Volume final de df_investments_clean : {df_investments_clean.shape}")
display(df_investments_clean.head(10))

- Suppression des doublons pour éviter de surestimer l'influence d'un investisseur dans votre futur calcul de prestige

- Suppression des colonnes temporelles système (created_at, updated_at) qui ne servent pas à la modélisation métier.

-On ne garde que le triplet funding_round_id, funded_object_id et investor_object_id pour la préparation de graphs

## 4. Nettoyage acquisition.csv

In [ ]:
# Sélection des colonnes essentielles pour le marquage du succès
df_acq_clean = acquisitions[['acquired_object_id', 'acquired_at']].copy()

# Suppression de la seule ligne manquante sur l'ID de la cible
df_acq_clean.dropna(subset=['acquired_object_id'], inplace=True)

# Conversion au format Datetime
df_acq_clean['acquired_at'] = pd.to_datetime(df_acq_clean['acquired_at'], errors='coerce')

# Suppression des doublons (Une startup ne peut être acquise qu'une fois)
df_acq_clean.drop_duplicates(subset=['acquired_object_id'], inplace=True)

print(f"Volume final de df_acq_clean : {df_acq_clean.shape}")

- Nous allons utiliser ce fichier uniquement pour créer un dictionnaire de succès. Si une startup apparaît ici, elle est marquée comme is_success = 1 dans notre dataset final.

- Suppression des colonnes inutiles (source_url, source_description, created_at).

- Conversion de acquired_at en datetime.

# Données prêtes:

- df_objects_clean : L'identité des startups.

- df_rounds_clean : L'historique des financements.

- df_investments_clean : Le réseau des investisseurs.

- df_acq_clean : Le registre des succès.


Nous allons maintenant passer à à la fusion de nos informations nettoyéesdans un DataFrame nommé df_final. Celui ci nous servira à l'analyse Eploratoire (EDA) et à l'entrainement de nos modèles.

In [ ]:
# 1. Distribution des Top 10 Secteurs (df_objects_clean)
plt.figure(figsize=(12, 6))
top_categories = df_objects_clean['category_code'].value_counts().head(10)
sns.barplot(x=top_categories.values, y=top_categories.index, palette='viridis')
plt.title('Top 10 des Secteurs d\'Activité (Après Nettoyage)')
plt.xlabel('Nombre d\'entreprises')
plt.ylabel('Secteur')
plt.show()

## Remarque sur le graphique :

- Le graphique montre une domination nette des secteurs liés au Software, au Web et à l'Entreprise.

- Malgré un gros nettoyage  ayant réduit le volume à environ 64 000 entreprises, la distribution reste représentative de l'économie numérique.

- Cette forte concentration suggère que le secteur d'activité sera une variable catégorielle déterminante (Feature) pour nos modèles de Machine Learning.


In [ ]:
# 2. Visualisation des Outliers Financiers (Fichier Funding Rounds)
plt.figure(figsize=(10, 5))
sns.boxplot(x=df_rounds_clean['raised_amount_usd'])
plt.xscale('log') # Échelle log car l'écart entre 0 et 3.8 milliards est trop grand
plt.title('Distribution des Montants Levés (Échelle Logarithmique)')
plt.xlabel('Montant en USD (Log)')
plt.show()

## Remarque sur le graphique :

- L'utilisation d'une échelle logarithmique est indispensable pour visualiser la donnée tant les écarts sont massifs (de quelques milliers à 3,8 milliards USD).

- Le boxplot révèle une multitude d'outliers (valeurs aberrantes) situés bien au-delà de la moustache supérieure. Cela confirme l'asymétrie (skewness) de la donnée financière.

- Ce graphique justifie mathématiquement notre choix d'utiliser la médiane ($\approx 1,6 \text{M USD}$) plutôt que la moyenne ($\approx 7,9 \text{M USD}$) pour l'imputation, car la moyenne est artificiellement tirée vers le haut par les "méga-levées"55.

In [ ]:
# 3. Répartition des types de financement
plt.figure(figsize=(10, 6))
round_types = df_rounds_clean['funding_round_type'].value_counts()
sns.countplot(data=df_rounds_clean, y='funding_round_type', order=round_types.index, palette='magma')
plt.title('Répartition des Types de Levées de Fonds')
plt.show()

## Remarque sur le graphique :  

- Les rounds de type Seed et Series-A sont les plus fréquents, suivis par les investissements "Angel"

- Le dataset capture majoritairement des startups en phase de démarrage. Pour notre modèle, la transition vers des Series-B ou C sera un signal fort de survie et de succès potentiel.

- Cette distribution nous incite à traiter le funding_round_type comme une variable ordinale lors du préprocessing.

## Fusion des données nettoyées :

In [ ]:
# 1. Agrégation des levées de fonds par startup
# On calcule le montant total levé et le nombre de rounds pour chaque entreprise
df_rounds_agg = df_rounds_clean.groupby('object_id').agg({
    'raised_amount_usd': 'sum',
    'funding_round_type': 'count'
}).rename(columns={
    'raised_amount_usd': 'total_funding_usd',
    'funding_round_type': 'funding_rounds_count'
}).reset_index()

# 2. Fusion de la table Objects avec les données financières
# Utilisation d'un 'left join' pour garder toutes les boîtes nettoyées
df_final = pd.merge(df_objects, df_rounds_agg, left_on='id', right_on='object_id', how='left')

# 3. Marquage des succès via la table Acquisitions
# On vérifie si l'ID de la startup est présent dans la liste des acquisitions
df_final = pd.merge(df_final, df_acq_clean[['acquired_object_id', 'acquired_at']],
                    left_on='id', right_on='acquired_object_id', how='left')

# 4. Finalisation de la variable cible 'is_success'
# Une startup a réussi si elle est déjà marquée 'acquired'/'ipo' OU si elle est dans df_acq
df_final['is_success'] = df_final['is_success'].fillna(0)
df_final.loc[df_final['acquired_at'].notnull(), 'is_success'] = 1

# 5. Nettoyage final post-fusion
df_final['total_funding_usd'] = df_final['total_funding_usd'].fillna(0)
df_final['funding_rounds_count'] = df_final['funding_rounds_count'].fillna(0)

# Suppression des colonnes techniques de jointure devenues inutiles
df_final.drop(columns=['object_id', 'acquired_object_id', 'acquired_at'], inplace=True)

print(f"Volume du dataset consolidé : {df_final.shape}")
display(df_final.head())

## Rapport sur le Merge :

- objects.csv est devenu Le Squelette : Il fournit la structure de base (ID, Nom, Secteur, Pays, Date de fondation). C'est la table "maître" qui porte l'identité de chaque startup.


- funding_rounds.csv sert de carburant : Il a été "compressé" via une agrégation. Au lieu d'avoir plusieurs lignes par startup pour chaque levée, on a désormais deux indicateurs uniques par entreprise : total_funding_usd (somme de l'argent reçu) et funding_rounds_count (nombre de fois où elle a levé des fonds).



- acquisitions.csv sert de dictionnaire de vérification. Si l'ID d'une startup était présent dans ce fichier, nous avons forcé sa variable is_success à 1, même si son statut initial dans objects n'était pas à jour.

- Enfin investments.csv sera utilisé pour la partie Modèle non-supervisé pour calcluer l'influence des investisseur sans polluer la structure de la table de classification supervisée.

Les données de financement ont été transformées par une opération de groupby, permettant de convertir un historique transactionnel en variables statistiques clés : le montant total levé et le nombre de rounds de financement.

Étiquetage de la cible : La variable cible is_success a été binarisée (0 ou 1) en croisant le statut initial avec le registre des acquisitions, garantissant une vérité terrain fiable pour l'entraînement des futurs modèles.

- Le dataset final comprend 64 099 observations et 12 variables prédictives.

- Cohérence métier : L'analyse des premières lignes (échantillon) confirme la robustesse de la fusion : les startups à succès (ex: FriendFeed) sont correctement identifiées avec leurs indicateurs financiers et relationnels respectifs.

- Prêt pour la modélisation : Cette table unique réunit désormais les dimensions temporelles, géographiques, sectorielles et financières nécessaires pour répondre à notre problématique de prédiction.

# Modèle non-supervisé

In [ ]:
# Matrice de corrélation
plt.figure(figsize=(10, 8))
# On ne garde que les colonnes numériques
sns.heatmap(df_final.select_dtypes(include=[np.number]).corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice de Corrélation - Variables finales')
plt.show()

In [ ]:

# Calculer le nombre d'investisseurs uniques par startup
investor_counts = df_investments_clean.groupby('funded_object_id')['investor_object_id'].nunique().reset_index()
investor_counts.columns = ['id', 'investor_count']

# Intégrer ce "Prestige Score" dans notre dataset final
df_final = pd.merge(df_final, investor_counts, on='id', how='left')

# Remplacer les NaN par 0 (les startups sans investisseurs listés)
df_final['investor_count'] = df_final['investor_count'].fillna(0)

print("Variable 'investor_count' ajoutée avec succès.")
print(df_final[['name', 'investor_count']].head())

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans

# 1. Copie de sauvegarde pour la modélisation
df_ml = df_final.copy()

# 2. Encodage des variables textuelles (Catégories et Pays)
le = LabelEncoder()
df_ml['category_code'] = le.fit_transform(df_ml['category_code'].astype(str))
df_ml['country_code'] = le.fit_transform(df_ml['country_code'].astype(str))

# 3. Sélection des features pour le Clustering et le ML
# On retire les IDs et les noms qui ne servent pas au calcul
features = ['category_code', 'country_code', 'funding_total_usd', 'relationships', 'total_funding_usd', 'investor_count']
X = df_ml[features]

# 4. Standardisation (Obligatoire pour K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Application de K-Means (3 clusters pour la lisibilité)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_ml['cluster_profile'] = kmeans.fit_predict(X_scaled)

# Visualisation des profils
print("Répartition des startups par profil :")
print(df_ml['cluster_profile'].value_counts())

# Analyse rapide des groupes
cluster_analysis = df_ml.groupby('cluster_profile')[['total_funding_usd', 'investor_count', 'is_success']].mean()
print("\nAnalyse moyenne par cluster :")
display(cluster_analysis)

In [ ]:
from sklearn.decomposition import PCA

# 1. Graphique de la PCA (Visualisation des clusters)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df_ml['cluster_profile'], palette='viridis', s=60)
plt.title('Visualisation des Clusters (Projection PCA)')
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
plt.legend(title='Profil')
plt.show()

In [ ]:
cluster_summary = df_ml.groupby('cluster_profile')[['total_funding_usd', 'investor_count', 'relationships']].mean()

In [ ]:
cluster_summary_norm = (cluster_summary - cluster_summary.min()) / (cluster_summary.max() - cluster_summary.min())
cluster_summary_norm.plot(kind='bar', figsize=(12, 6))
plt.title('Comparaison des caractéristiques moyennes par Cluster (Normalisé)')
plt.ylabel('Score (0 à 1)')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Vérification de la pertinence des clusters par rapport au succès
check_clusters = df_ml.groupby('cluster_profile')['is_success'].mean() * 100
print("Pourcentage de succès par profil de startup :")
print(check_clusters)

In [ ]:
# --- ÉTAPE : Identification des Licornes Potentielles ---

# 1. Filtrer les startups qui n'ont pas encore "réussi" (pas encore rachetées/IPO)
# On se concentre sur celles qui sont dans le cluster d'élite (Cluster 2)
potential_unicorns = df_ml[(df_ml['is_success'] == 0) & (df_ml['cluster_profile'] == 2)].copy()

# 2. Création d'un score combiné (Unicorn Score)
# On normalise les variables pour qu'elles aient le même poids
from sklearn.preprocessing import MinMaxScaler
scaler_score = MinMaxScaler()

cols_to_score = ['total_funding_usd', 'investor_count', 'relationships']
potential_unicorns['unicorn_score'] = scaler_score.fit_transform(potential_unicorns[cols_to_score]).sum(axis=1)

# 3. Sélection du Top 5
top_5_unicorns = potential_unicorns.sort_values(by='unicorn_score', ascending=False).head(5)

print("TOP 5 - Licornes Potentielles (Cibles d'investissement prioritaires) :")
display(top_5_unicorns[['name', 'category_code', 'total_funding_usd', 'investor_count', 'unicorn_score']])

# 4. Graphique du Top 5 pour le rapport
plt.figure(figsize=(12, 6))
sns.barplot(data=top_5_unicorns, x='unicorn_score', y='name', color='gold')
plt.title('Top 5 des Startups à fort potentiel (Unicorn Status)')
plt.xlabel('Score de Potentiel (Finance/Réseau/Relations)')
plt.ylabel('Nom de la Startup')
plt.show()

# Modèle supervisé

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Définition des colonnes (Page 67 du cours)
num_features = ['funding_total_usd', 'relationships', 'total_funding_usd', 'investor_count', 'cluster_profile']
cat_features = ['category_code', 'country_code']

# 2. Preprocessing Pipeline (Bonnes pratiques Module 5)
preprocessor = ColumnTransformer(
    transformers=[
        # Standarisation par défaut avant de tester d'autres approches (Page 106)
        ('num', StandardScaler(), num_features),
        # OrdinalEncoder recommandé pour les pipelines (Page 106)
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features)
    ])

# 3. Pipeline complète : Preprocessing + Modèle
# scale_pos_weight est utilisé pour gérer le déséquilibre des classes
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(n_estimators=100, learning_rate=0.1, scale_pos_weight=10, random_state=42))
])

# 4. Split Train/Test
X = df_ml[num_features + cat_features]
y = df_ml['is_success']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 5. Entraînement
clf.fit(X_train, y_train)

# 6. Évaluation (Page 82 du cours)
y_pred = clf.predict(X_test)
print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}")
print("\nMatrice de Confusion :\n", confusion_matrix(y_test, y_pred))
print("\nRapport de Classification :\n", classification_report(y_test, y_pred))

In [ ]:
# Extraction de l'importance des variables depuis la pipeline
importances = clf.named_steps['classifier'].feature_importances_
feature_names = num_features + cat_features

# Création du graphique
plt.figure(figsize=(10, 6))
pd.Series(importances, index=feature_names).sort_values().plot(kind='barh', color='skyblue')
plt.title('Quels facteurs prédisent le succès ? (Feature Importance)')
plt.show()

In [ ]:

from sklearn.metrics import RocCurveDisplay

plt.figure(figsize=(8, 6))
RocCurveDisplay.from_estimator(clf, X_test, y_test)
plt.title('Courbe ROC - Performance du Modèle')
plt.plot([0, 1], [0, 1], 'k--') # Ligne de base (aléatoire)
plt.show()

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

plt.figure(figsize=(8, 6))
PrecisionRecallDisplay.from_estimator(clf, X_test, y_test)
plt.title('Courbe Precision-Recall (Gestion du déséquilibre)')
plt.show()

In [ ]:
# Extraction des noms des colonnes après transformation
# Note: On récupère les noms des features définies plus tôt
feature_names = num_features + cat_features
importances = clf.named_steps['classifier'].feature_importances_

plt.figure(figsize=(10, 6))
feat_importances = pd.Series(importances, index=feature_names)
feat_importances.nlargest(10).sort_values().plot(kind='barh', color='teal')
plt.title('Top des Facteurs Déterminants du Succès')
plt.show()

In [5]:
import joblib
# On enregistre la pipeline complète (XGBoost + Preprocessing)
joblib.dump(clf, 'models/unicorn_model.pkl')
# On enregistre le modèle de clustering
joblib.dump(kmeans, 'models/unicorn_clusters.pkl')

NameError: name 'clf' is not defined